<a href="https://colab.research.google.com/github/zachrenwick/data_viz/blob/master/DY_Testing_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DY Analysis Objectives

*   Trying to match revenue (high level and down to DY user ID)
*   Trying to match ID counts (DY export to GA)

*   Maybe there are missing user IDs?

# Setup/Library Installations

In [ ]:
!pip install --quiet duckdb
!pip install --quiet jupysql 
!pip install --quiet duckdb-engine

In [ ]:
# Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import numpy as np
import pandas as pd

import duckdb
# No need to import sqlalchemy or duckdb_engine
#  JupySQL will use SQLAlchemy to auto-detect the driver needed based on your connection string!

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

project = 'atz-data-ecomm-prd' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.disable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [ ]:
%sql duckdb:///:memory:

# Raw DY Export

In [ ]:
# Analyze raw DY export
sql1 = """
SELECT dy_id, cast(date as string) as date, ifnull(revenue/100, 0) as dy_revenue, 'CAN' AS country  
FROM `aritzia-ga.da_sandbox.lj_dy_can_raw` 

UNION ALL 

SELECT dy_id, cast(date as string) as date, ifnull(revenue/100, 0) as dy_revenue, 'US' AS country 
FROM `aritzia-ga.da_sandbox.lj_dy_us_raw` """

dy_raw = client.query(sql1).to_dataframe()
dy_raw

,dy_id,date,dy_revenue,country
0,-3391210268628542722,2023-01-09,0.0,US
1,5994291354810449076,2023-01-09,0.0,US
2,5534924184229845757,2023-01-09,0.0,US
3,2760706809473955972,2023-01-09,0.0,US
4,-3292131119779631862,2023-01-09,0.0,US
...,...,...,...,...
1714797,-2711166729191394885,2023-01-15,0.0,CAN
1714798,22518253140754073,2023-01-15,0.0,CAN
1714799,-6106880899769215186,2023-01-15,0.0,CAN
1714800,-2026619624491118414,2023-01-15,0.0,CAN


# Google Analytics Export

In [ ]:
# Bring in GA revenue by customDimension DY_ID, visit_id, and date
sql2 = """
--cte for revenue
with ga_mat as
(
select
  fullVisitorId                          as full_visitor_id,
  visitId                                as ga_visit_id,
  date                                   as session_date_pt,
  ifnull(localrevenue/1000000,0)         as ga_revenue
FROM `aritzia-ga.da_datasets.ga_sessions_materialized_*` 
WHERE
date BETWEEN '2023-01-09' AND '2023-01-15'
),

--cte for DY user IDs
dy_ids as 
(select
 fullVisitorId                   as full_visitor_id,
 visitId                         as ga_visit_id,
 parse_date("%Y%m%d", date)      as session_date_pt,
(SELECT value FROM UNNEST(hits.customDimensions) WHERE index = 121) AS ga_dy_id
from `aritzia-ga.141585386.ga_sessions_*`, 
unnest(hits) as hits
where
 _table_suffix not like '%2017%'
 and _table_suffix between format_date('%Y%m%d', '2023-01-09') and format_date('%Y%m%d', '2023-01-15')
 and eventinfo.eventCategory = 'dy smart action' AND REGEXP_CONTAINS(eventinfo.eventAction, r'prso-150')
group by full_visitor_id, ga_visit_id, ga_dy_id, session_date_pt
)


select
ga_mat.ga_visit_id,
dy_ids.ga_dy_id,
sum(ga_mat.ga_revenue) as ga_revenue,
cast(dy_ids.session_date_pt as string) as date
from dy_ids
left outer join ga_mat
  on  dy_ids.ga_visit_id = ga_mat.ga_visit_id
  and dy_ids.full_visitor_id = ga_mat.full_visitor_id
  and dy_ids.session_date_pt = ga_mat.session_date_pt
where dy_ids.ga_dy_id is not null
group by ga_visit_id, ga_dy_id, dy_ids.session_date_pt """

ga_dy_ids = client.query(sql2).to_dataframe()
ga_dy_ids

,ga_visit_id,ga_dy_id,ga_revenue,date
0,1673337543,-815151337613815676,0.0,2023-01-09
1,1673337566,3985685908119427806,0.0,2023-01-09
2,1673337433,6922032878049892953,0.0,2023-01-09
3,1673337595,-4841369425956898619,0.0,2023-01-09
4,1673337584,-6444650858941465124,0.0,2023-01-09
...,...,...,...,...
1084748,1673797392,-2526519144468233228,0.0,2023-01-15
1084749,1673835993,-6863485658644765500,0.0,2023-01-15
1084750,1673811287,2121195675273418071,0.0,2023-01-15
1084751,1673784371,4305441485958147122,0.0,2023-01-15


## Google Analytics Summary

In [ ]:
%sql ga_distinct_counts << SELECT count(distinct(ga_dy_id)) as dy_id_count, count(distinct(ga_visit_id)) as ga_visit_count, cast(sum(ga_revenue) as numeric) as ga_revenue  FROM ga_dy_ids
ga_distinct_counts

,dy_id_count,ga_visit_count,ga_revenue
0,762164,410435,7548567.400


# Raw DY vs Google Analytics Comparison

In [ ]:
%sql summary_check << SELECT 'google_analytics' as source, count(distinct(ga_dy_id)) as dy_id_count, cast(sum(ga_revenue) as numeric) as revenue FROM ga_dy_ids union all SELECT 'DY_raw' as source, count(distinct(dy_id)) as dy_id_count, cast(sum(dy_revenue) as numeric) as revenue FROM dy_raw
summary_check

,source,dy_id_count,revenue
0,google_analytics,762164,7548567.400
1,DY_raw,814920,7560761.340


### Raw DY vs GA (id and revenue check) by day

In [ ]:
# query using duckdb SQL on a dataframe 
duckdb_con = duckdb.connect()

sql5 = """
with ga as 
  (SELECT 'google_analytics' as source, date, count(distinct(ga_dy_id)) as dy_id_count, cast(sum(ga_revenue) as numeric) as revenue 
   FROM ga_dy_ids group by date), 
   
dy as (  
   SELECT 'DY_raw' as source, date, count(distinct(dy_id)) as dy_id_count, cast(sum(dy_revenue) as numeric) as revenue 
   FROM dy_raw group by date)

select coalesce(ga.date, dy.date) as date,
       ga.dy_id_count as ga_dy_count,
       dy.dy_id_count as dy_dy_count,
       abs(ga.dy_id_count - dy.dy_id_count) as dy_count_diff,
       ga.revenue as ga_revenue,
       dy.revenue as dy_revenue,
       abs(ga.revenue - dy.revenue) as revenue_diff
from ga
full outer join dy
 on dy.date = ga.date 
order by date

"""
summary_check2 = duckdb_con.execute(sql5).df()
summary_check2

,date,ga_dy_count,dy_dy_count,dy_count_diff,ga_revenue,dy_revenue,revenue_diff
0,2023-01-09,70,70,0,0.00,0.00,0.00
1,2023-01-10,155082,166428,11346,1239271.91,1240039.71,767.80
2,2023-01-11,148855,159860,11005,1208806.30,1201851.46,6954.84
3,2023-01-12,164238,175515,11277,1385602.16,1373824.21,11777.95
4,2023-01-13,153729,164160,10431,1244008.21,1252194.19,8185.98
5,2023-01-14,158023,167436,9413,1201122.02,1205013.98,3891.96
6,2023-01-15,164918,175031,10113,1269756.80,1287837.79,18080.99


# Unused queries

In [ ]:
# query using duckdb SQL on a dataframe 
duckdb_con = duckdb.connect()

sql4 = """
SELECT * FROM 
  (SELECT 'google_analytics' as source, date, count(distinct(ga_dy_id)) as dy_id_count, cast(sum(ga_revenue) as numeric) as revenue 
   FROM ga_dy_ids group by date 
   
   union all 
   
   SELECT 'DY_raw' as source, date, count(distinct(dy_id)) as dy_id_count, cast(sum(dy_revenue) as numeric) as revenue 
   FROM dy_raw group by date
   
) order by date
"""
summary_check = duckdb_con.execute(sql4).df()
summary_check

,source,date,dy_id_count,revenue
0,google_analytics,2023-01-09,70,0.00
1,DY_raw,2023-01-09,70,0.00
2,google_analytics,2023-01-10,155082,1239271.91
3,DY_raw,2023-01-10,166428,1240039.71
4,google_analytics,2023-01-11,148855,1208806.30
5,DY_raw,2023-01-11,159860,1201851.46
6,DY_raw,2023-01-12,175515,1373824.21
7,google_analytics,2023-01-12,164238,1385602.16
8,google_analytics,2023-01-13,153729,1244008.21
9,DY_raw,2023-01-13,164160,1252194.19
